In [18]:
from koalas import DataFrame
from example import chess

In [19]:
# Generate the DataFrame given the chess results, which are already in the form of records.
df = DataFrame.from_records(list(chess('TineBerger', 2023, 10)))
df[2]

KeyError: 'games'

In [ ]:
# Get the breakdown of how many games have been won, lost and drawn.
(
    df
        .group('Outcome')
        .apply('Count', len, 'Color')
        .select('Outcome', 'Count')
)

Outcome Count
------- -----
Win     1040 
Lose    999  
Draw    154  

In [ ]:
# Identify the most difficult players.
(
    df
        .filter('Outcome', 'Lose')
        .group('Opponent')
        .apply('Losses', len, 'Outcome')
        .select('Opponent', 'Losses')
        .sort('Losses', False)
        [:5]
)

Opponent    Losses
--------    ------
scoott33    4     
Fabrizio265 3     
dryenja     3     
drmaj       3     
Kaiidlu     3     

In [ ]:
# Identify which openings are most and least successful when playing as white.
(
    df
        .filter('Color', 'White')
        .apply('Opening', lambda moves: ' '.join(moves[:4]), 'Moves')
        .group('Opening')
        .apply('Count', len, 'Outcome')
        .apply('Win Rate', lambda outcomes: outcomes.count('Win') / len(outcomes), 'Outcome')                
        .apply('Win Percentage', lambda rate: round(100. * rate, 2), 'Win Rate')
        .sort('Win Percentage', False)
        .apply('Minimum', lambda count: count > 25, 'Count')
        .filter('Minimum', True)
        .select('Opening', 'Win Percentage', 'Count')
)

Opening                        Win Percentage Count
-------                        -------------- -----
1. Nc3 1... Nf6 2. e4 2... d6  64.58          48   
1. Nc3 1... d5 2. e4 2... d4   58.01          181  
1. Nc3 1... Nf6 2. e4 2... e5  57.58          66   
1. Nc3 1... Nf6 2. e4 2... g6  53.12          32   
1. Nc3 1... g6 2. e4 2... Bg7  50.85          59   
1. Nc3 1... d5 2. e4 2... dxe4 50.0           64   
1. Nc3 1... e5 2. e4 2... Nf6  50.0           62   
1. Nc3 1... c6 2. e4 2... d5   45.61          57   
1. Nc3 1... c5 2. e4 2... Nc6  44.68          47   
1. Nc3 1... e6 2. e4 2... d5   39.47          38   
1. Nc3 1... e5 2. e4 2... Nc6  36.21          58   